In [1]:
!pip install wandb pyyaml opencv-python;

In [2]:
import os
import yaml
import glob
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import wandb
from wandb.integration.keras import WandbCallback
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

2025-06-03 18:54:29.958745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 18:54:29.977227: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 18:54:29.982885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 18:54:29.997800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-03 18:54:30.773952: W tensorflow/compiler/tf2

In [3]:
wandb.login(key="017feefe0af6702cda76aab121ec71cf3a362fec")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: samutheman (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
dataset_yaml = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/data.yaml"
images_trainset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/train"
labels_trainset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/train"
images_valset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/val"
labels_valset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/val"
images_testset = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/images/test"
test_csv_path = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01/labels/test/bboxes_test.csv"


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
# --- Parameters (tweak these to your dataset) ---
IMG_SIZE    = 640
GRID_SIZE   = 10
NUM_CLASSES = 1   #
NUM_BOXES   = 3    # number of anchors per cell
OUTPUT_SHAPE = (GRID_SIZE, GRID_SIZE, NUM_BOXES * (5 + NUM_CLASSES))

In [7]:
def build_yolo_model(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = tf.keras.Input(shape=input_shape)

    def conv_bn_leaky(x, filters):
        x = layers.Conv2D(filters, 4, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        return layers.LeakyReLU(alpha=0.1)(x)

    x = conv_bn_leaky(inputs, 16)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 32)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 64)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 128)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 256)
    x = layers.MaxPool2D(2)(x)

    x = conv_bn_leaky(x, 512)
    x = layers.Dropout(0.3)(x)
    x = layers.MaxPool2D(2)(x)
    x = conv_bn_leaky(x, 512)

    # Final prediction conv
    x = layers.Conv2D(
        NUM_BOXES * (5 + NUM_CLASSES),
        kernel_size=1,
        padding='same',
        activation='sigmoid'  # or linear + custom loss
    )(x)

    print("Final conv shape:", x.shape)

    # Check shape compatibility
    output = layers.Reshape((GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))(x)
    print("Final conv output shape:", output.shape)

    return tf.keras.Model(inputs, output)


In [8]:
# --- YOLO loss with binary class ---
def yolo_loss(y_true, y_pred):
    # reshape to (..., S, S, B, 5 + C)
    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    obj_mask = true[..., 4:5]

    # localization losses
    xy_loss = tf.reduce_sum(obj_mask * tf.square(true[..., 0:2] - pred[..., 0:2]))
    wh_loss = tf.reduce_sum(obj_mask * tf.square(
        tf.sqrt(true[..., 2:4] + 1e-6) - tf.sqrt(pred[..., 2:4] + 1e-6)
    ))

    # objectness and class losses
    obj_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(true[..., 4:5], pred[..., 4:5])
    )
    class_bce = tf.keras.losses.binary_crossentropy(true[..., 5:], pred[..., 5:])
    class_bce = tf.expand_dims(class_bce, axis=-1)
    class_loss = tf.reduce_sum(obj_mask * class_bce)

    return xy_loss + wh_loss + obj_loss + class_loss

In [9]:
def compute_giou(y_true, y_pred):
    # Unpack
    y1_true, x1_true, y2_true, x2_true = tf.split(y_true, 4, axis=-1)
    y1_pred, x1_pred, y2_pred, x2_pred = tf.split(y_pred, 4, axis=-1)

    # Intersection
    x1 = tf.maximum(x1_true, x1_pred)
    y1 = tf.maximum(y1_true, y1_pred)
    x2 = tf.minimum(x2_true, x2_pred)
    y2 = tf.minimum(y2_true, y2_pred)

    intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)

    # Area
    area_true = (x2_true - x1_true) * (y2_true - y1_true)
    area_pred = (x2_pred - x1_pred) * (y2_pred - y1_pred)
    union = area_true + area_pred - intersection

    iou = intersection / (union + 1e-7)

    # Enclosing box
    x1_c = tf.minimum(x1_true, x1_pred)
    y1_c = tf.minimum(y1_true, y1_pred)
    x2_c = tf.maximum(x2_true, x2_pred)
    y2_c = tf.maximum(y2_true, y2_pred)

    enclose_area = (x2_c - x1_c) * (y2_c - y1_c)

    giou = iou - (enclose_area - union) / (enclose_area + 1e-7)
    return 1.0 - giou  # Loss = 1 - GIoU


In [10]:
def improved_yolo_loss(y_true, y_pred):
    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))

    obj_mask = true[..., 4:5]
    noobj_mask = 1.0 - obj_mask

    giou = compute_giou(true[..., 0:4], pred[..., 0:4])
    giou = tf.squeeze(giou, axis=-1) 
    giou_loss = tf.reduce_sum(obj_mask[..., 0] * giou)

    obj_loss = tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(true[..., 4:5], pred[..., 4:5])
    )

    noobj_loss = tf.reduce_sum(
        noobj_mask * tf.square(pred[..., 4:5])
    )

    class_bce = tf.keras.losses.binary_crossentropy(true[..., 5:], pred[..., 5:])
    class_loss = tf.reduce_sum(obj_mask * tf.expand_dims(class_bce, -1))

    return giou_loss + obj_loss + class_loss + 0.5 * noobj_loss


In [11]:
def area_difference_loss(y_true, y_pred):

    pred = tf.reshape(y_pred, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))
    true = tf.reshape(y_true, (-1, GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES))


    pred_area = pred[..., 2] * pred[..., 3] * pred[..., 4]
    true_area = true[..., 2] * true[..., 3] * true[..., 4]


    total_pred_area = tf.reduce_sum(pred_area)
    total_true_area = tf.reduce_sum(true_area)


    return tf.abs(total_pred_area - total_true_area)

In [12]:
def combined_yolo_area_loss(y_true, y_pred, area_weight=0.01):

    yolo = yolo_loss(y_true, y_pred)

    area = area_difference_loss(y_true, y_pred)

    return yolo + area_weight * area

In [13]:
def apply_nms(pred_grid, conf_thresh=0.3, iou_thresh=0.4):
    boxes = []
    scores = []

    for gy in range(GRID_SIZE):
        for gx in range(GRID_SIZE):
            for b in range(NUM_BOXES):
                pred = pred_grid[gy, gx, b]
                conf = pred[4]
                if conf < conf_thresh:
                    continue

                x, y, w, h = pred[:4]
                cx = (gx + x) / GRID_SIZE
                cy = (gy + y) / GRID_SIZE
                xmin = cx - w / 2
                ymin = cy - h / 2
                xmax = cx + w / 2
                ymax = cy + h / 2

                boxes.append([ymin, xmin, ymax, xmax])
                scores.append(conf)

    if not boxes:
        return [], []

    boxes = tf.constant(boxes, dtype=tf.float32)
    scores = tf.constant(scores, dtype=tf.float32)

    selected_indices = tf.image.non_max_suppression(
        boxes,
        scores,
        max_output_size=10,
        iou_threshold=iou_thresh,
        score_threshold=conf_thresh
    )

    selected_boxes = tf.gather(boxes, selected_indices).numpy()
    selected_scores = tf.gather(scores, selected_indices).numpy()

    return selected_boxes, selected_scores



In [14]:
def build_yolo_label_grid(label_path, grid_size, num_boxes, num_classes):
    label_grid = np.zeros((grid_size, grid_size, num_boxes, 5 + num_classes), dtype=np.float32)

    if not os.path.exists(label_path):
        return label_grid

    with open(label_path, "r") as f:
        for line in f:
            class_id, x, y, w, h = map(float, line.strip().split())
            gx = min(int(x * grid_size), grid_size - 1)
            gy = min(int(y * grid_size), grid_size - 1)
            cell_x = x * grid_size - gx
            cell_y = y * grid_size - gy

            anchor_idx = 0

            label_grid[gy, gx, anchor_idx, 0:5] = [cell_x, cell_y, w, h, 1.0]
            label_grid[gy, gx, anchor_idx, 5 + int(class_id)] = 1.0

    return label_grid

def prepare_image_label_paths(image_dir, label_dir, grid_size, num_boxes, num_classes):
    image_paths = sorted(glob.glob(os.path.join(image_dir, "*.jpg")))
    label_grids = []

    for img_path in image_paths:
        filename = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, f"{filename}.txt")
        label_grid = build_yolo_label_grid(label_path, grid_size, num_boxes, num_classes)
        label_grids.append(label_grid)

    return image_paths, np.array(label_grids, dtype=np.float32)


In [15]:
def parse_image_and_label(image_path, label_tensor, img_size, augment=False):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_size, img_size])
    img = img / 255.0

    if augment:
        img, label_tensor = apply_augmentations(img, label_tensor)

    img.set_shape([img_size, img_size, 3])
    label_tensor.set_shape([GRID_SIZE, GRID_SIZE, NUM_BOXES, 5 + NUM_CLASSES])

    print("Label tensor shape example:", label_tensor.shape)
    
    return img, label_tensor


def make_yolo_dataset(image_paths, label_tensors, img_size, batch_size=4, shuffle=True, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((image_paths, label_tensors))
    
    if shuffle:
        ds = ds.shuffle(buffer_size=len(image_paths))

    ds = ds.map(lambda p, l: parse_image_and_label(p, l, img_size, augment),
                num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds



In [16]:
def apply_augmentations(img, label):
    # Random horizontal flip
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        label = flip_label_left_right(label)

    # Random vertical flip
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_up_down(img)
        label = flip_label_up_down(label)

    # Random brightness
    if tf.random.uniform(()) > 0.5:
        img = tf.image.random_brightness(img, max_delta=0.1)

    # Random contrast
    if tf.random.uniform(()) > 0.5:
        img = tf.image.random_contrast(img, lower=0.9, upper=1.1)

    # Random 90° rotation
    k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    img = tf.image.rot90(img, k=k)
    label = rotate_label(label, k)

    # Random crop + resize back to original
    if tf.random.uniform(()) > 0.5:
        crop_size = tf.cast(
            tf.random.uniform([], 0.8, 1.0) * tf.cast(tf.shape(img)[:2], tf.float32),
            tf.int32
        )
        img = tf.image.random_crop(img, size=[crop_size[0], crop_size[1], 3])
        img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

    return img, label


In [17]:
def flip_label_left_right(label):
    # Flip the label grid horizontally
    flipped = tf.reverse(label, axis=[1])  # axis 1 = width/grid-x

    # Get the x offset and update it: x' = 1 - x
    x = flipped[..., 0]
    flipped = tf.tensor_scatter_nd_update(
        flipped,
        tf.where(flipped[..., 4] > 0.5),
        tf.concat([
            tf.expand_dims(1.0 - tf.gather_nd(x, tf.where(flipped[..., 4] > 0.5)), axis=-1),  # new x
            tf.gather_nd(flipped[..., 1:], tf.where(flipped[..., 4] > 0.5))  # rest unchanged
        ], axis=-1)
    )

    return flipped

    
def flip_label_up_down(label):
    # Flip the grid vertically (axis 0 = row/height dction)
    label = tf.reverse(label, axis=[0])

    # Adjust the y offset: new_y = 1.0 - y
    y_offset = label[..., 1]
    label = tf.tensor_scatter_nd_update(
        label,
        tf.where(label[..., 4] > 0.5),
        tf.concat([
            tf.expand_dims(label[..., 0][label[..., 4] > 0.5], -1),
            tf.expand_dims(1.0 - y_offset[label[..., 4] > 0.5], -1),
            label[..., 2:][label[..., 4] > 0.5]
        ], axis=-1)
    )

    return label

In [18]:
def rotate_label(label, k):
    original_shape = tf.shape(label)
    has_batch = (len(label.shape) == 5)

    if not has_batch:
        label = tf.expand_dims(label, 0)  # make shape [1, H, W, A, P]

    B, H, W, A, P = tf.shape(label)[0], tf.shape(label)[1], tf.shape(label)[2], tf.shape(label)[3], tf.shape(label)[4]

    # Flatten A and P into channels for rotation
    label_flat = tf.reshape(label, [B, H, W, A * P])
    label_rotated_flat = tf.image.rot90(label_flat, k=k)

    # Swap H and W if needed
    new_H = tf.cond(tf.logical_or(tf.equal(k, 1), tf.equal(k, 3)), lambda: W, lambda: H)
    new_W = tf.cond(tf.logical_or(tf.equal(k, 1), tf.equal(k, 3)), lambda: H, lambda: W)

    label_rotated = tf.reshape(label_rotated_flat, [B, new_H, new_W, A, P])

    # Adjust x and y
    x = label_rotated[..., 0]
    y = label_rotated[..., 1]

    def no_rot(): return x, y
    def rot90(): return y, 1.0 - x
    def rot180(): return 1.0 - x, 1.0 - y
    def rot270(): return 1.0 - y, x

    new_x, new_y = tf.case([
        (tf.equal(k, 0), no_rot),
        (tf.equal(k, 1), rot90),
        (tf.equal(k, 2), rot180),
        (tf.equal(k, 3), rot270),
    ], default=no_rot)

    other_params = label_rotated[..., 2:]
    updated = tf.concat([
        tf.expand_dims(new_x, axis=-1),
        tf.expand_dims(new_y, axis=-1),
        other_params
    ], axis=-1)

    if not has_batch:
        updated = tf.squeeze(updated, axis=0)

    return updated


In [19]:
def evaluate_loss(model, test_ds, batch_size=4):
    preds_all = model.predict(test_ds, verbose=1)

    total_loss = 0.0
    total_area_loss = 0.0
    num_batches = 0

    for i, (images, labels) in enumerate(test_ds):
        start = i * batch_size
        end = start + images.shape[0]
        preds = tf.convert_to_tensor(preds_all[start:end])

        area_loss = area_difference_loss(labels, preds).numpy()
        loss = combined_yolo_area_loss(labels, preds).numpy()

        total_area_loss += area_loss
        total_loss += loss
        num_batches += 1

    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    avg_area_loss = total_area_loss / num_batches if num_batches > 0 else 0

    print(f"Average loss on test set: {avg_loss:.4f}")
    print(f"Average area loss on test set: {avg_area_loss:.4f}")

    wandb.log({
        "test_combined_loss": avg_loss,
        "test_area_difference_loss": avg_area_loss
    })

    return avg_loss


In [20]:
def create_label_grids_from_df(df, image_dir, grid_size, num_boxes, num_classes):
    df = df.copy()
    grouped = df.groupby("image")
    
    image_paths = []
    label_grids = []

    for img_name, group in grouped:
        img_path = os.path.join(image_dir, img_name)
        image_paths.append(img_path)

        label_grid = np.zeros((grid_size, grid_size, num_boxes, 5 + num_classes), dtype=np.float32)

        for _, row in group.iterrows():
            cx, cy = row["x_center"], row["y_center"]
            bw, bh = row["bb_width"], row["bb_height"]
            class_id = 0  # or use `row["class_id"]` if multi-class

            gx = min(int(cx * grid_size), grid_size - 1)
            gy = min(int(cy * grid_size), grid_size - 1)
            cell_x = cx * grid_size - gx
            cell_y = cy * grid_size - gy

            anchor_idx = 0  # assign to first anchor
            label_grid[gy, gx, anchor_idx, 0:5] = [cell_x, cell_y, bw, bh, 1.0]
            label_grid[gy, gx, anchor_idx, 5 + class_id] = 1.0

        label_grids.append(label_grid)

    return image_paths, np.array(label_grids, dtype=np.float32)


In [21]:
def add_image_size_to_df(df, image_size_dict):
    df = df.copy()
    df['original_img_width'] = df['image'].map(lambda img: image_size_dict[img][0])
    df['original_img_height'] = df['image'].map(lambda img: image_size_dict[img][1])
    return df

def prepare_bboxes(df):
    df = df.copy()
    df["x1"] = df["x1"] / df["original_img_width"]
    df["x2"] = df["x2"] / df["original_img_width"]
    df["y1"] = df["y1"] / df["original_img_height"]
    df["y2"] = df["y2"] / df["original_img_height"]

    df["x_center"] = (df["x1"] + df["x2"]) / 2
    df["y_center"] = (df["y1"] + df["y2"]) / 2
    df["bb_width"] = df["x2"] - df["x1"]
    df["bb_height"] = df["y2"] - df["y1"]
    
    return df

def get_image_size_dict(folder_path):
    image_size_dict = {}
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.jpg'):
            img = Image.open(os.path.join(folder_path, filename))
            image_size_dict[filename] = img.size

    return image_size_dict

In [22]:
bboxes_test= None
bboxes_df= None

In [23]:
bboxes_df = pd.read_csv(test_csv_path)
img_size_dict_test = get_image_size_dict(images_testset)
bboxes_test = add_image_size_to_df(bboxes_df, img_size_dict_test)
bboxes_test = prepare_bboxes(bboxes_test)

In [24]:
bboxes_test.head()

,image,x1,y1,x2,y2,user,day,month,year,hour,minute,type,original_img_width,original_img_height,x_center,y_center,bb_width,bb_height
0,DSC00976_0.jpg,0.612667,0.376,0.661333,0.4315,Tyr,12,12,2016,9,45,boom,3000,2000,0.637000,0.40375,0.048667,0.0555
1,DSC00976_0.jpg,0.652333,0.349,0.717000,0.4165,Tyr,12,12,2016,9,45,boom,3000,2000,0.684667,0.38275,0.064667,0.0675
2,DSC00976_0.jpg,0.626333,0.403,0.675667,0.4800,Tyr,12,12,2016,9,45,boom,3000,2000,0.651000,0.44150,0.049333,0.0770
3,DSC00976_0.jpg,0.576000,0.190,0.651333,0.2885,Tyr,12,12,2016,9,45,boom,3000,2000,0.613667,0.23925,0.075333,0.0985
4,DSC00976_0.jpg,0.569667,0.151,0.652333,0.2640,Tyr,12,12,2016,9,45,boom,3000,2000,0.611000,0.20750,0.082667,0.1130


In [25]:
test_image_paths, test_labels = create_label_grids_from_df(
    bboxes_test,
    images_testset,    
    GRID_SIZE,
    NUM_BOXES,
    NUM_CLASSES
)
test_ds = make_yolo_dataset(
    test_image_paths,
    test_labels,
    IMG_SIZE,
    batch_size=4,
    shuffle=False
)

Label tensor shape example: (10, 10, 3, 6)


2025-06-03 18:54:35.355006: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13294 MB memory:  -> device: 0, name: NVIDIA A16, pci bus id: 0000:2a:00.0, compute capability: 8.6


In [26]:
def save_test_dataset_with_images(dataset, image_paths, output_dir="saved_test_set_5"):
    os.makedirs(output_dir, exist_ok=True)
    image_dir = os.path.join(output_dir, "images")
    os.makedirs(image_dir, exist_ok=True)

    tfrecord_path = os.path.join(output_dir, "test_dataset.tfrecord")
    writer = tf.io.TFRecordWriter(tfrecord_path)

    for i, (img_tensor, label_tensor) in enumerate(dataset):
        for b in range(img_tensor.shape[0]):
            image = img_tensor[b].numpy()
            label = label_tensor[b].numpy()
            filename = os.path.basename(image_paths[i * img_tensor.shape[0] + b])
            name, _ = os.path.splitext(filename)

            # Save image
            pil_img = Image.fromarray((image * 255).astype(np.uint8))
            pil_img.save(os.path.join(image_dir, filename))

            # Serialize image + label
            image_bytes = tf.io.encode_jpeg(tf.cast(image * 255, tf.uint8)).numpy()
            label_bytes = tf.io.serialize_tensor(label).numpy()

            feature = {
                "image_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes])),
                "label_raw": tf.train.Feature(bytes_list=tf.train.BytesList(value=[label_bytes])),
                "filename": tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode()]))
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

    writer.close()
    print(f"Saved TFRecord: {tfrecord_path}")
    print(f"Saved {len(image_paths)} images to: {image_dir}")


In [27]:
#save_test_dataset_with_images(test_ds, test_image_paths)

In [28]:
train_image_paths, train_labels = prepare_image_label_paths(
    images_trainset, labels_trainset, GRID_SIZE, NUM_BOXES, NUM_CLASSES
)

val_image_paths, val_labels = prepare_image_label_paths(
    images_valset, labels_valset, GRID_SIZE, NUM_BOXES, NUM_CLASSES
)


In [29]:
train_ds = make_yolo_dataset(train_image_paths, train_labels, IMG_SIZE, batch_size=4, augment=True)
val_ds   = make_yolo_dataset(val_image_paths, val_labels, IMG_SIZE, batch_size=4, shuffle=False)

Label tensor shape example: (10, 10, 3, 6)
Label tensor shape example: (10, 10, 3, 6)


In [30]:
run = wandb.init(project="Own CNN", name="FinalModelNewLoss")

In [31]:
model = build_yolo_model()
model.compile(optimizer='adam', loss=improved_yolo_loss)
model.summary()

Final conv shape: (None, 10, 10, 18)
Final conv output shape: (None, 10, 10, 3, 6)


/opt/conda/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 640, 640, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 640, 640, 16)   │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 640, 640, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 640, 640, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 320, 320, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 320, 320, 32)   │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 320, 320, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 320, 320, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 160, 160, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 160, 160, 64)   │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 160, 160, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 160, 160, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 80, 80, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 80, 80, 128)    │       131,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 80, 80, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 80, 80, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 40, 40, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 40, 40, 256)    │       524,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 40, 40, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 40, 40, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 20, 20, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 20, 20, 512)    │     2,097,152 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 7,003,858 (26.72 MB)

 Trainable params: 7,000,818 (26.71 MB)

 Non-trainable params: 3,040 (11.88 KB)

In [ ]:
batch_size = 4  

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[WandbCallback(save_graph=False, save_model=False)]
)
run_id = run.id

model_filename = f"saved_cnn_models/full_model_general_big_V1.0_{run_id}.keras"

model.save(model_filename)

artifact = wandb.Artifact(name="cnn-model", type="model")
artifact.add_file(model_filename)
run.log_artifact(artifact)


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


Epoch 1/15


I0000 00:00:1748976881.723710   74339 service.cc:146] XLA service 0x7effac0086f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748976881.723764   74339 service.cc:154]   StreamExecutor device (0): NVIDIA A16, Compute Capability 8.6
2025-06-03 18:54:42.100532: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-03 18:54:42.468102: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2025-06-03 18:54:56.549893: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:762] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


   2/2715 ━━━━━━━━━━━━━━━━━━━━ 3:21 74ms/step - loss: 1665.4434   

I0000 00:00:1748976899.511670   74339 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 917/2715 ━━━━━━━━━━━━━━━━━━━━ 2:03 69ms/step - loss: 57431.3555

In [ ]:
evaluate_loss(model, test_ds)

In [ ]:
def draw_nms_boxes(image, label_grid, pred_boxes):
    img = (image * 255).astype(np.uint8).copy()
    h, w, _ = img.shape
    cell_h, cell_w = h // GRID_SIZE, w // GRID_SIZE

    for gy in range(GRID_SIZE):
        for gx in range(GRID_SIZE):
            for b in range(NUM_BOXES):
                if label_grid[gy, gx, b, 4] > 0.5:
                    x, y, bw, bh = label_grid[gy, gx, b, :4]
                    cx = int((gx + x) * cell_w)
                    cy = int((gy + y) * cell_h)
                    bw = int(bw * w)
                    bh = int(bh * h)
                    cv2.rectangle(img, (cx - bw // 2, cy - bh // 2), (cx + bw // 2, cy + bh // 2), (0, 255, 0), 2)

    for box in pred_boxes:
        ymin, xmin, ymax, xmax = box
        x1 = int(xmin * w)
        y1 = int(ymin * h)
        x2 = int(xmax * w)
        y2 = int(ymax * h)
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

    return img



In [ ]:
for i, (image_tensor, label_tensor) in enumerate(test_ds.take(10)):
    image_batch = image_tensor.numpy()
    label_batch = label_tensor.numpy()  

    for b in range(image_batch.shape[0]):
        img_np = image_batch[b]
        label_np = label_batch[b]
        
        pred = model.predict(img_np[np.newaxis])[0] 

        nms_boxes, scores = apply_nms(pred, conf_thresh=0.3, iou_thresh=0.4)

        vis_img = draw_nms_boxes(img_np, label_np, nms_boxes)  # custom function below

        plt.imshow(vis_img)
        plt.axis('off')
        plt.title(f"Green = GT, Red = Pred (sample {i}-{b})")
        plt.show()


In [ ]:
run.finish()